In [ ]:
from os import urandom  
from Crypto.Cipher import AES
from ecdsa import SigningKey, NIST384p
from hashlib import sha256

sk = SigningKey.generate(curve=NIST384p)        # Secret key
vk = sk.verifying_key                           # Public key
idx = 0

In [ ]:
class Server:
    def __init__(self):
        self.key = urandom(16)                  # 256 bit
        pass
    
    def key_rotation(self):
        global idx
        idx = (idx + 16) % 256
        self.key = urandom(16)
        print("current Server key: ", self.key.hex())   # In ra key hiện tại
        
    def decrypt(self, enc_msg):
        key = self.key
        nonce = enc_msg[:12]
        ct = enc_msg[12:-16]
        tag = enc_msg[-16:]

        cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
        msg = cipher.decrypt_and_verify(ct, tag)
        return msg

In [ ]:
examples = [
    "admin = True",
    "admin = False",
    
]

def str2byte(s):
    b = s.encode('utf-8')
    
    # In thông tin
    print(f"Chuỗi gốc: {s}")
    print(f"Độ dài (ký tự): {len(s)}")
    print(f"Độ dài (byte): {len(b)}")
    print(f"Hex tương ứng: {b.hex()}")
    
    b_from_hex = bytes.fromhex(b.hex())
    return b_from_hex


for i in examples:
    k = str2byte(i)

Chuỗi gốc: admin = True
Độ dài (ký tự): 12
Độ dài (byte): 12
Hex tương ứng: 61646d696e203d2054727565
Chuỗi gốc: admin = False
Độ dài (ký tự): 13
Độ dài (byte): 13
Hex tương ứng: 61646d696e203d2046616c7365


In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import binascii, struct, math

def get_gcm_keystream_blocks(key, nonce, num_blocks=1):
    ecb = AES.new(key, AES.MODE_ECB)
    blocks = []
    for i in range(1, num_blocks+1):
        ctr_block = nonce + struct.pack(">I", i)
        blocks.append(ecb.encrypt(ctr_block))
    return b"".join(blocks)

def aes_gcm_encrypt(key, nonce, plaintext, aad=b""):
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    cipher.update(aad)
    ct, tag = cipher.encrypt_and_digest(plaintext)
    return ct, tag

def aes_gcm_decrypt_verify(key, nonce, ciphertext, tag, aad=b""):
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    cipher.update(aad)
    return cipher.decrypt_and_verify(ciphertext, tag)

k1 = get_random_bytes(16)                     # AES-128 key for k1 (replace if needed)
k2 = get_random_bytes(16)                     # AES-128 key for k2 (replace if needed)
nonce = get_random_bytes(12)                  # 96-bit nonce (common for GCM)
p1 = b"admin = Falso skadmin = True sdf"      # example plaintext (you can change)
aad = b""                                     # optional associated data (kept empty here)
# ---------------------------------------------------------------------------

# Prepare
num_blocks = math.ceil(len(p1) / 16)

S1_full = get_gcm_keystream_blocks(k1, nonce, num_blocks)
S2_full = get_gcm_keystream_blocks(k2, nonce, num_blocks)
S1 = S1_full[:len(p1)]
S2 = S2_full[:len(p1)]

# Compute p2 so CTR-based ciphertexts match
# C = P1 xor S1  and want C = P2 xor S2  =>  P2 = P1 xor S1 xor S2
p2 = bytes(a ^ b ^ c for a, b, c in zip(p1, S1, S2))

# Compute CTR-based ciphertexts (direct XOR) for verification
ctr_ct_from_p1 = bytes(a ^ b for a, b in zip(p1, S1))
ctr_ct_from_p2 = bytes(a ^ b for a, b in zip(p2, S2))

# Encrypt with AES-GCM using library
ct1, tag1 = aes_gcm_encrypt(k1, nonce, p1, aad=aad)
ct2, tag2 = aes_gcm_encrypt(k2, nonce, p2, aad=aad)

# Verification attempts
try:
    rec1 = aes_gcm_decrypt_verify(k1, nonce, ct1, tag1, aad=aad)
    ok1 = True
except Exception as e:
    rec1 = None
    ok1 = False
    err1 = str(e)

try:
    rec2 = aes_gcm_decrypt_verify(k2, nonce, ct2, tag2, aad=aad)
    ok2 = True
except Exception as e:
    rec2 = None
    ok2 = False
    err2 = str(e)

# Cross-verify: try to verify ct1 with tag2 under k1 (should fail if keys differ)
try:
    aes_gcm_decrypt_verify(k1, nonce, ct1, tag2, aad=aad)
    cross_ok = True
except Exception as e:
    cross_ok = False
    cross_err = str(e)

# Print outputs
print("=== Inputs ===")
print("k1:", binascii.hexlify(k1).decode())
print("k2:", binascii.hexlify(k2).decode())
print("nonce:", binascii.hexlify(nonce).decode())
print("p1 (hex):", p1.hex(), " / repr:", repr(p1))
print()

print("=== Keystream blocks ===")
for i in range(num_blocks):
    ctr_block = nonce + struct.pack(">I", i+1)
    print(f"CTR block {i+1} :", binascii.hexlify(ctr_block).decode())
    s1_block = S1_full[i*16:(i+1)*16]
    s2_block = S2_full[i*16:(i+1)*16]
    print(f"  S1 block {i+1} :", s1_block.hex())
    print(f"  S2 block {i+1} :", s2_block.hex())

print("\nS1 (truncated to p1 len):", S1.hex())
print("S2 (truncated to p1 len):", S2.hex())
print()

print("=== Derived p2 ===")
print("p2 (hex):", p2.hex(), " / repr:", repr(p2))
print()

print("=== CTR-based ciphertexts (direct XOR) ===")
print("CTR ciphertext from p1,S1:", ctr_ct_from_p1.hex())
print("CTR ciphertext from p2,S2:", ctr_ct_from_p2.hex())
print("CTR-based equal?:", ctr_ct_from_p1 == ctr_ct_from_p2)
print()

print("=== AES-GCM (library) outputs ===")
print("ct1 (hex):", ct1.hex())
print("tag1 (hex):", tag1.hex())
print("ct2 (hex):", ct2.hex())
print("tag2 (hex):", tag2.hex())
print("Library ciphertexts equal?:", ct1 == ct2)
print("CTR-based ct equals library ct1?:", ctr_ct_from_p1 == ct1)
print("CTR-based ct equals library ct2?:", ctr_ct_from_p2 == ct2)
print()

print("=== Verification results ===")
print("Verify ct1 with k1/tag1:", "OK" if ok1 else f"FAIL ({err1})", "; recovered:", rec1)
print("Verify ct2 with k2/tag2:", "OK" if ok2 else f"FAIL ({err2})", "; recovered:", rec2)
print("Verify ct1 with k1 but tag2 (cross-check):", "OK" if cross_ok else f"FAIL ({cross_err})")

=== Inputs ===
k1: 2eddaf5452e1cac79550204cfdd25c5c
k2: e97ea6598a46e12e965cd156e626a734
nonce: e5df70e30b8c19a96701cdd2
p1 (hex): 61646d696e203d2046616c736f20736b61646d696e203d205472756520736466  / repr: b'admin = Falso skadmin = True sdf'

=== Keystream blocks ===
CTR block 1 : e5df70e30b8c19a96701cdd200000001
  S1 block 1 : 4ef2a4c22e10362d0a75761d7cb7b520
  S2 block 1 : c957ced1aaf348b78ec82cbf2b6e0119
CTR block 2 : e5df70e30b8c19a96701cdd200000002
  S1 block 2 : 8b4c55981353bcdd8793266af20021b3
  S2 block 2 : 72d3a31947646f26683b284e2b9fe7f2

S1 (truncated to p1 len): 4ef2a4c22e10362d0a75761d7cb7b5208b4c55981353bcdd8793266af20021b3
S2 (truncated to p1 len): c957ced1aaf348b78ec82cbf2b6e011972d3a31947646f26683b284e2b9fe7f2

=== Derived p2 ===
p2 (hex): e6c1077aeac343bac2dc36d138f9c75298fb9be83a17eedbbbda7b41f9eca227  / repr: b"\xe6\xc1\x07z\xea\xc3C\xba\xc2\xdc6\xd18\xf9\xc7R\x98\xfb\x9b\xe8:\x17\xee\xdb\xbb\xda{A\xf9\xec\xa2'"

=== CTR-based ciphertexts (direct XOR) ===
CTR ciphert

In [6]:
import re
import sys
import socket
from os import urandom
from Crypto.Cipher import AES


# ----- Helper stubs -----
def parse_key(buf: bytes) -> bytes:
    m = re.search(rb"Key\s*\(in hex\):\s*([0-9a-fA-F]+)", buf)
    if not m:
        raise ValueError("Could not parse key")
    return bytes.fromhex(m.group(1).decode())

def recv_until(s: socket.socket, delim: bytes) -> bytes:
    data = b""
    while not data.endswith(delim):
        chunk = s.recv(1024)
        if not chunk:
            break
        data += chunk
    return data

def sendline(s: socket.socket, line: str):
    s.sendall(line.encode() + b"\n")

# ----- Cryptography stubs -----
def gcm_keystream_blocks(key: bytes, nonce: bytes, nblocks: int):
    # Trả về danh sách nblocks block giả lập
    block = AES.new(key, AES.MODE_ECB).encrypt(b"\x00"*16)
    blocks = [block]*nblocks
    return blocks, block

def make_dual_valid(K_server: bytes, K_flag: bytes, nonce: bytes, C0: bytes):
    # Trả về C và tag giả lập, ở thực tế cần thực hiện tạo ciphertext dual-valid
    cipher = AES.new(K_server, AES.MODE_GCM, nonce=nonce)
    C, tag = cipher.encrypt_and_digest(C0)
    return C, tag

def parse_sig(buf: bytes):
    m = re.search(rb"Signature \(in hex\):\s*([0-9a-fA-F]+)", buf)
    if not m:
        raise ValueError("Could not parse signature")
    return m.group(1).decode()

# ----- Build ciphertext -----
def build_enc_msg(K_server, K_flag, idx=16):
    nonce = urandom(12)

    S1_blocks, EK1_J0 = gcm_keystream_blocks(K_server, nonce, 2)
    S2_blocks, EK2_J0 = gcm_keystream_blocks(K_flag, nonce, 2)
    S1 = b"".join(S1_blocks)
    S2 = b"".join(S2_blocks)
    Delta = bytes(a ^ b for a, b in zip(S1, S2))

    p2_block0 = b"admin = True" + b" "*(16 - len(b"admin = True"))
    p1_block0 = bytes(a ^ b for a, b in zip(p2_block0, Delta[:16]))
    if b"admin = True" in p1_block0:
        p2_block0 = b"admin = True" + b"\x00"*(16 - len(b"admin = True"))
        p1_block0 = bytes(a ^ b for a, b in zip(p2_block0, Delta[:16]))

    p1_block1 = b"admin = False" + b" "*(16 - len(b"admin = False"))
    p2_block1 = bytes(a ^ b for a, b in zip(p1_block1, Delta[16:32]))
    if b"admin = False" in p2_block1:
        p1_block1 = b"admin = False" + b"\x00"*(16 - len(b"admin = False"))
        p2_block1 = bytes(a ^ b for a, b in zip(p1_block1, Delta[16:32]))

    P1 = p1_block0 + p1_block1
    C0 = bytes(m ^ k for m, k in zip(P1, S1))

    C, tag = make_dual_valid(K_server, K_flag, nonce, C0)

    m1 = AES.new(K_server, AES.MODE_GCM, nonce=nonce).decrypt_and_verify(C, tag)
    m2 = AES.new(K_flag, AES.MODE_GCM, nonce=nonce).decrypt_and_verify(C, tag)
    assert (b'admin = False' in m1[idx:idx+16]) and (b'admin = True' not in m1)
    assert (b'admin = True' in m2[:idx]) and (b'admin = False' not in m2)

    enc_msg = nonce + C + tag
    return enc_msg

# ----- Main logic -----
def main():
    HOST = "crypto1.cscv.vn"
    PORT = 1337
    PROMPT = b"choice: "
    
    
    print("[*] Connecting to {}:{}...".format(HOST, PORT))
    s = socket.create_connection((HOST, PORT))
    buf = recv_until(s, PROMPT)
    s.sendline(s, b"1")  # Rotate Server key

    print("[*] Rotating Server (idx -> 16)...")
    sendline(s, "1")
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    K_server_a = parse_key(buf)
    print(f"[+] Server key 1: {K_server_a.hex()}")

    print("[*] Rotating FlagKeeper (idx -> 0)...")
    sendline(s, "2")
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    K_flag = parse_key(buf)
    print(f"[+] FlagKeeper key: {K_flag.hex()}")

    print("[*] Rotating Server again (idx -> 16)...")
    sendline(s, "1")
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    K_server = parse_key(buf)
    print(f"[+] Server key 2: {K_server.hex()}")

    print("[*] Building dual-valid ciphertext...")
    enc_msg = build_enc_msg(K_server, K_flag, idx=16)
    enc_hex = enc_msg.hex()
    print(f"[+] Ciphertext: {enc_hex[:60]}...")

    print("[*] Requesting signature...")
    sendline(s, "3")
    recv_until(s, b"encrypted message (in hex):")
    sendline(s, enc_hex)
    buf = recv_until(s, PROMPT)
    sys.stdout.write(buf.decode(errors="ignore"))
    sig_hex = parse_sig(buf)
    print(f"[+] Got signature: {sig_hex[:60]}...")

    print("[*] Getting flag...")
    sendline(s, "4")
    recv_until(s, b"encrypted message (in hex):")
    sendline(s, enc_hex)
    recv_until(s, b"signature (in hex):")
    sendline(s, sig_hex)
    buf = recv_until(s, PROMPT)
    out = buf.decode(errors="ignore")
    sys.stdout.write(out)
    m = re.search(r"Flag:\s*(.*)", out)
    if m:
        print("\n[+] 🎉 " + m.group(0))
    s.close()

if __name__ == "__main__":
    main()


[*] Connecting to crypto1.cscv.vn:1337...


AttributeError: 'socket' object has no attribute 'sendline'